In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [60]:
# collections data
csv_file = "resources/Mia_objects.csv"

In [61]:
db = pd.read_csv(csv_file, index_col='Unnamed: 0')
db.head()

,accession_number,artist,classification,continent,country,culture,dated,department,id,room,title
0,10.1,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c.1888-89,Prints and Drawings,0,Not on View,"Air, from the series The Four Elements"
1,10.2,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,1,Not on View,"Earth, from the series The Four Elements"
2,10.3,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,2,G352,"Fire, from the series The Four Elements"
3,10.4,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,3,Not on View,"Water, from the series The Four Elements"
4,13.29,Walter Shirlaw,Drawings,North America,United States,NaN,19th century,Prints and Drawings,4,Not on View,Montana Indian Reservation I


In [62]:
# strip "P" from prints & drawings
db['accession_number'] = db['accession_number'].str.lstrip('P.')

# split out first half of accession number
db['accession_year'] = db['accession_number'].str.split('.', n=1, expand=True)[0]

In [72]:
# add loan column
db['loan'] = db['accession_year'].str.extract(r'([L])', expand=True)

# strip "L" from loaned accession_year column
db['accession_year'] = db['accession_year'].str.lstrip('L')

In [74]:
db.head()

,accession_number,artist,classification,continent,country,culture,dated,department,id,room,title,accession_year,loan
0,10.1,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c.1888-89,Prints and Drawings,0,Not on View,"Air, from the series The Four Elements",10,NaN
1,10.2,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,1,Not on View,"Earth, from the series The Four Elements",10,NaN
2,10.3,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,2,G352,"Fire, from the series The Four Elements",10,NaN
3,10.4,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,NaN,c. 1888-89,Prints and Drawings,3,Not on View,"Water, from the series The Four Elements",10,NaN
4,13.29,Walter Shirlaw,Drawings,North America,United States,NaN,19th century,Prints and Drawings,4,Not on View,Montana Indian Reservation I,13,NaN


In [75]:
db['accession_year'].value_counts()

99        7600
2013      3929
2003      3582
98        3548
2007      3120
2004      2989
2002      2858
96        2628
2001      2491
2015      2476
97        2273
2005      2178
74        1877
2000      1845
2012      1781
2014      1722
2018      1709
95        1681
82        1670
90        1669
2006      1628
77        1623
81        1586
2010      1574
94        1474
2016      1466
75        1393
84        1347
2008      1319
2017      1306
          ... 
4,366        1
3,289        1
10,550       1
3,110        1
910          1
6,992        1
10,967       1
18,467       1
2,495        1
7,887        1
8,733        1
10,299       1
15,280       1
2,282        1
14,653       1
9,904        1
12,947       1
14,373       1
16,449       1
11,013       1
12,111       1
945          1
10,062       1
21,299       1
7,540        1
8,577        1
7,754        1
3,811        1
22,050       1
19,466       1
Name: accession_year, Length: 21950, dtype: int64